<a href="https://colab.research.google.com/github/lilxmx/Artificial-Intelligence-Map/blob/main/Deep%20Learning%20In%20PyTorch/autograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


#自动求导机制介绍
---------------------------------

``torch.autograd`` 是一个可以推动神经网络训练的自动微分引擎.在这一节，你将会对自动求导如何帮助神经网络有一个概念性的理解.  

##背景

神经网络是在某些输入数据上执行嵌套函数的集合，这些函数有被权重（weight）和偏置（biases）所组成的参数所定义，这些参数在pytorch中由张量（tensor）所保存。

### 训练一个神经网络分为两步：

- 前向传播（Forward Propagation）: 在前向传播中，神经网络对正确的输出做出最好的猜测。它通过其每个函数运行输入数据以进行猜测。
- 反向传播（Backward Propagation）: 在反向传播中，神经网络根据猜测中的误差调整神经网络参数。它通过从输出向后遍历，收集关于函数参数（梯度）的误差导数，并使用梯度下降优化参数来做到这一点。对于更多详细介绍，观看视频[3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)




##PyTorch实战

从一个简单的训练过程开始，例如，从`torchvision`模块中加载一个预训练的resnet18模型，创建一个随机张量去表示一个简单的三通道图像，高度和宽度为64的图像，相应的标签用一些随机值初始化，在模型中，标签的shape（1,1000）

### 注意
本教程仅适用于 CPU，不适用于 GPU（即使将张量移至 CUDA）。



In [ ]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

接下来，将输入数据输入模型，通过每一层的运算得到预测结果，这就是前向传播过程




In [ ]:
prediction = model(data) # forward pass
print(prediction.shape)

torch.Size([1, 1000])


我们使用模型进行预测得到预测值，并且利用相应的标签去计算误差（`loss`）。  
下一步是通过网络反向传播误差
当我们在误差张量（`tensor`）上使用`.backward`函数时，反向传播开始。
自动求导（Autograd），即计算模型每个参数的梯度并存储在参数的`.grad`属性中。 




In [ ]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

接下来，加载一个SGD优化器，学习率为 0.01，动量为 0.9。我们在优化器中注入模型的所有参数。




In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

最后，使用`.step()`方法去开始梯度下降。优化器通过每个参数的`.grad`属性调整其值。




In [ ]:
optim.step() #gradient descent

此时，您已经拥有了训练神经网络所需的全部知识。下面的一节是自动求导工作的细节，根据自己需要阅读。



## Autograd: 自动求导机制
PyTorch 中所有神经网络的核心是 `autograd` 包。 我们先简单介绍一下这个包，然后训练第一个简单的神经网络。

`autograd`包为张量上的所有操作提供了自动求导。 它是一个在运行时定义的框架，这意味着反向传播是根据你的代码来确定如何运行，并且每次迭代可以是不同的。

### 张量（Tensor）

`torch.Tensor`是这个包的核心类。如果设置 `.requires_grad` 为 True，那么将会追踪所有对于该张量的操作。 当完成计算后通过调用 `.backward()`，自动计算所有的梯度， 这个张量的所有梯度将会自动积累到 `.grad` 属性。

要阻止张量跟踪历史记录，可以调用.detach()方法将其与计算历史记录分离，并禁止跟踪它将来的计算记录。

为了防止跟踪历史记录（和使用内存），可以将代码块包装在`with torch.no_grad()：`中。 在评估模型时特别有用，因为模型可能具有`requires_grad = True`的可训练参数，但是我们不需要梯度计算。

在自动梯度计算中还有另外一个重要的类`Function`.

`Tensor and Function are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a .grad_fn attribute that references a Function that has created the Tensor (except for Tensors created by the user - their grad_fn is None).`

`Tensor` 和 `Function`互相连接并生成一个有向无环图(Directed Acyclic Graph，DAG)，它表示和存储了完整的计算历史。 每个张量都有一个`.grad_fn`属性，该属性即创建该Tensor的Function，就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

我们已经知道PyTorch使用有向无环图DAG记录计算的全过程，那么DAG是怎样建立的呢？DAG的节点是Function对象，边表示数据依赖，从输出指向输入。 每当对Tensor施加一个运算的时候，就会产生一个Function对象，它产生运算的结果，记录运算的发生，并且记录运算的输入。Tensor使用.grad_fn属性记录这个计算图的入口。反向传播过程中，autograd引擎会按照逆序，通过Function的backward依次计算梯度。 

如果需要计算导数，你可以在Tensor上调用`.backward()`。 如果Tensor是一个标量（即它包含一个元素数据）则不需要为`backward()`指定任何参数， 但是如果它有更多的元素，你需要指定一个`gradient` 参数来匹配张量的形状。

译者注：在其他的文章中你可能会看到说将Tensor包裹到Variable中提供自动梯度计算，Variable 这个在0.41版中已经被标注为过期了，现在可以直接使用Tensor，官方文档在这里： (https://pytorch.org/docs/stable/autograd.html#variable-deprecated)

具体的后面会有详细说明

In [ ]:
import torch

### 代码示例
创建一个Tensor并设置requires_grad=True:

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn) # 可以看到，这里手动创建一个张量（非计算得到），其grad_fn=None


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


再做一下运算操作：

In [ ]:
y = x + 2
print(y)
print(y.grad_fn)


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


注意`x`是直接创建的，所以它没有`grad_fn`, 而`y`是通过一个加法操作创建的，所以它有一个为`<AddBackward>`的`grad_fn`。

像`x`这种直接创建的称为叶子节点，叶子节点对应的`grad_fn`是None。

In [ ]:
print(x.is_leaf, y.is_leaf) # True False


True False


再来点复杂度运算操作：

In [ ]:
z = y * y * 3
out = z.mean()
print(z, out)


tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


可以通过.requires_grad_()来用in-place的方式改变requires_grad属性：


In [ ]:
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)


False
True


### autograd 自动求梯度
深度学习模型的训练就是不断更新权值，权值的更新需要求解梯度，梯度在模型训练中是至关重要的。Pytorch提供自动求导系统，我们不需要手动计算梯度，只需要搭建好前向传播的计算图，然后根据Pytorch中的autograd方法就可以得到所有张量的梯度。 PyTorch中，所有神经网络的核心是autograd包。autograd包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义（define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的。

#### torch.autograd.backward

```
torch.autograd.backward(tensors,grad_tensors=None,retain_grad=None,create_graph=False)

```



- 功能：自动求取梯度
- tensors: 用于求导的张量，如loss
- retain_graph : 保存计算图；由于pytorch采用动态图机制，在每一次反向传播结束之后，计算图都会释放掉。如果想继续使用计算图，就需要设置参数retain_graph为True
- create_graph : 创建导数计算图，用于高阶求导，例如二阶导数、三阶导数等
- grad_tensors：多梯度权重；当有多个loss需要去计算梯度的时候，就要设计各个loss之间的权重比例

#### torch.autograd.grad

```
torch.autograd.grad(outputs,
                    inputs,
                    grad_outputs=None,
                    retain_graph=None,
                    create_graph=False)

```
功能：计算并返回outputs对inputs的梯度
- outputs：用于求导的张量，如loss
- inputs：需要梯度的张量，如w
- create_graph：创建导数计算图，用于高阶求导
- retain_graph：保存计算图
- grad_outputs：多梯度权重


#### 链式法则


数学上，如果有一个函数值和自变量都为向量的函数 $\vec{y}=f(\vec{x})$, 那么 $\vec{y}$ 关于 $\vec{x}$ 的梯度就是一个雅可比矩阵（Jacobian matrix）:  
$$ J=\left(\begin{array}{ccc} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\ \vdots & \ddots & \vdots\\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{array}\right) $$ 
而torch.autograd这个包就是用来计算一些雅克比矩阵的乘积的。例如，如果 $v$ 是一个标量函数的 $l=g\left(\vec{y}\right)$ 的梯度： $$ v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right) $$ 那么根据链式法则我们有 $l$ 关于 $\vec{x}$ 的雅克比矩阵就为: $$ v J=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right) \left(\begin{array}{ccc} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\ \vdots & \ddots & \vdots\\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{array}\right)=\left(\begin{array}{ccc}\frac{\partial l}{\partial x_{1}} & \cdots & \frac{\partial l}{\partial x_{n}}\end{array}\right) $$

注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。

#### 代码示例

In [ ]:
import torch
torch.manual_seed(10)  #用于设置随机数

w = torch.tensor([1.], requires_grad=True)    #创建叶子张量，并设定requires_grad为True，因为需要计算梯度；
x = torch.tensor([2.], requires_grad=True)    #创建叶子张量，并设定requires_grad为True，因为需要计算梯度；

a = torch.add(w, x)    #执行运算并搭建动态计算图 a = w + x
b = torch.add(w, 1)    # b = w + 1
y = torch.mul(a, b)    # y = a * b  即 (w+x)*(w+1)

y.backward(retain_graph=True)   
print(w.grad)   #输出为tensor([5.])


tensor([5.])


从代码中可以发现对y求导使用的是y.backward()方法，也就是张量中的类方法。我们上面介绍的是torch.autograd中的backward()。这两个方法之间有什么联系呢？ 通过pycharm中的断点调试，可以发现y.backward()是Tensor.py中的一个类方法的函数。这个函数只有一行代码，就是调用torch.autograd.backward()。

In [ ]:
def backward(self, gradient=None, retain_graph=None, create_graph=False):
    torch.autograd.backward(self, gradient, retain_graph, create_graph)


从代码调试中可以知道张量中的backward()方法实际直接调用了torch.autograd中的backward()。 backward()中有一个retain_grad参数，它是用来保存计算图的，如果还想执行一次反向传播 ，必须将retain_grad参数设置为retain_grad=True，否则代码会报错。因为如果没有retain_grad=True，每进行一次backward之后，计算图都会被清空，没法再进行一次backward()操作。

In [ ]:
y.backward()   
print(w.grad)   #输出为tensor([5.])


tensor([10.])


#### 关于y.backward()
为什么在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor?
> 简单来说就是为了避免向量（甚至更高维张量）对张量求导，而转换成标量对张量求导。举个例子，假设形状为 m x n 的矩阵 X 经过运算得到了 p x q 的矩阵 Y，Y 又经过运算得到了 s x t 的矩阵 Z。那么按照前面讲的规则，dZ/dY 应该是一个 s x t x p x q 四维张量，dY/dX 是一个 p x q x m x n的四维张量。问题来了，怎样反向传播？怎样将两个四维张量相乘？？？这要怎么乘？？？就算能解决两个四维张量怎么乘的问题，四维和三维的张量又怎么乘？导数的导数又怎么求，这一连串的问题，感觉要疯掉……   
为了避免这个问题，我们不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。所以必要时我们要把张量通过将所有张量的元素加权求和的方式转换为标量，举个例子，假设y由自变量x计算而来，w是和y同形的张量，**则y.backward(w)的含义是：先计算l = torch.sum(y * w)，则l是个标量，然后求l对自变量x的导数。**

实例

In [ ]:
import torch
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)


tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


现在 `z` 不是一个标量，所以在调用`backward`时需要传入一个和`z`同形的权重向量进行加权求和得到一个标量。

In [ ]:
w = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(w,retain_graph=True) # torch.sum(z * w) 
print(x.grad)


tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [ ]:
torch.sum(z * w)

tensor(2.4680, grad_fn=<SumBackward0>)

注意，x.grad是和x同形的张量。

再来看看中断梯度追踪的例子：

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
    
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True


True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


可以看到，上面的y2是没有grad_fn而且y2.requires_grad=False的，而y3是有grad_fn的。如果我们将y3对x求梯度的话会是多少呢？

In [ ]:
y3.backward()
print(x.grad)

tensor(2.)


为什么是2呢？$ y_3 = y_1 + y_2 = x^2 + x^3$，当 $x=1$ 时 $\frac {dy_3} {dx}$ 不应该是5吗？事实上，由于 $y_2$ 的定义是被torch.no_grad():包裹的，所以与 $y_2$ 有关的梯度是不会回传的，只有与 $y_1$ 有关的梯度才会回传，即 $x^2$ 对 $x$ 的梯度。  
上面提到，y2.requires_grad=False，所以不能调用 `y2.backward()`，会报错：  


```
RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

```
此外，如果我们想要修改`tensor`的数值，但是又不希望被`autograd`记录（即不会影响反向传播），那么我么可以对`tensor.data`进行操作。



In [ ]:
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)


tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])


#### 注意事项
- 梯度不自动清零，如果不清零梯度会累加，所以需要在每次梯度后人为清零。
- 依赖于叶子结点的结点，requires_grad默认为True。
- 叶子结点不可执行in-place，因为其他节点在计算梯度时需要用到叶子节点，所以叶子地址中的值不得改变否则会是其他节点求梯度时出错。所以叶子节点不能进行原位计算。
- 注意在y.backward()时，如果y是标量量，则不需要为backward()传⼊入任何参数；否则，需要传⼊一个与y同形的Tensor。

--------------


